# **YOLO11 Unified Dataset Training (Kaggle Version)**

This notebook:
1. **Merges** Road Lane and BDD100K datasets into a unified dataset (17 classes)
2. **Trains** YOLO11 models on the merged dataset

**Unified Classes (17 total)**

| ID | Class | Source |
|----|-------|--------|
| 0-5 | Lane types (divider, dotted, double, random, road-sign, solid) | Road Lane |
| 6-16 | Traffic objects (bike, bus, car, drivable area, motor, person, rider, traffic light, traffic sign, train, truck) | BDD100K |

## **Setup & Installation**

In [ ]:
!pip install ultralytics -q

In [ ]:
import os
import shutil
import yaml
import time
from pathlib import Path
from typing import Dict, Optional
from collections import defaultdict
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import torch
from ultralytics import YOLO

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## **Dataset Paths (Kaggle)**

In [ ]:
# Kaggle input paths
BDD100K_DIR = Path('/kaggle/input/bdd100ka')
ROAD_LANE_DIR = Path('/kaggle/input/roadlaneds/Road Lane.v2i.yolo26')

# Output paths (writable in Kaggle)
UNIFIED_DIR = Path('/kaggle/working/unified')
RUNS_DIR = Path('/kaggle/working/runs')

# Verify input datasets exist
print("Checking input datasets...")
print(f"BDD100K:    {BDD100K_DIR} - {'✓ Found' if BDD100K_DIR.exists() else '✗ NOT FOUND'}")
print(f"Road Lane:  {ROAD_LANE_DIR} - {'✓ Found' if ROAD_LANE_DIR.exists() else '✗ NOT FOUND'}")

# List contents to verify structure
if BDD100K_DIR.exists():
    print(f"\nBDD100K contents: {list(BDD100K_DIR.iterdir())[:5]}")
if ROAD_LANE_DIR.exists():
    print(f"Road Lane contents: {list(ROAD_LANE_DIR.iterdir())[:5]}")

## **Step 1: Merge Datasets**

Merge Road Lane (6 lane type classes) and BDD100K (11 traffic classes) into a unified dataset.

In [ ]:
# BDD100K class ID remapping
# Original: ['bike', 'bus', 'car', 'drivable area', 'lane', 'motor', 'person', 'rider', 'traffic light', 'traffic sign', 'train', 'truck']
# IDs:        0       1      2        3            4        5        6         7            8              9            10       11

BDD100K_CLASS_REMAP: Dict[int, Optional[int]] = {
    0: 6,    # bike -> 6
    1: 7,    # bus -> 7
    2: 8,    # car -> 8
    3: 9,    # drivable area -> 9
    4: None, # lane -> SKIP (replaced by Road Lane's specific lane types)
    5: 10,   # motor -> 10
    6: 11,   # person -> 11
    7: 12,   # rider -> 12
    8: 13,   # traffic light -> 13
    9: 14,   # traffic sign -> 14
    10: 15,  # train -> 15
    11: 16,  # truck -> 16
}

UNIFIED_CLASS_NAMES = [
    # Road Lane classes (0-5)
    'divider-line', 'dotted-line', 'double-line',
    'random-line', 'road-sign-line', 'solid-line',
    # BDD100K classes (6-16)
    'bike', 'bus', 'car', 'drivable area', 'motor',
    'person', 'rider', 'traffic light', 'traffic sign', 'train', 'truck'
]

LANE_CLASSES = list(range(0, 6))
TRAFFIC_CLASSES = list(range(6, 17))

print(f"Total unified classes: {len(UNIFIED_CLASS_NAMES)}")
print(f"Lane types: {UNIFIED_CLASS_NAMES[:6]}")
print(f"Traffic objects: {UNIFIED_CLASS_NAMES[6:]}")

In [ ]:
def remap_bdd100k_label(label_path: Path, output_path: Path) -> bool:
    """Remap BDD100K label file to unified class IDs."""
    lines_out = []
    
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            
            class_id = int(parts[0])
            new_class_id = BDD100K_CLASS_REMAP.get(class_id)
            
            if new_class_id is None:
                continue  # Skip 'lane' class
            
            parts[0] = str(new_class_id)
            lines_out.append(' '.join(parts))
    
    if lines_out:
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            f.write('\n'.join(lines_out) + '\n')
        return True
    return False


def copy_road_lane_data(source_dir: Path, dest_dir: Path, split: str) -> Dict[str, int]:
    """Copy Road Lane data (no remapping needed)."""
    stats = {'images': 0, 'labels': 0, 'skipped': 0}
    
    src_images = source_dir / split / 'images'
    src_labels = source_dir / split / 'labels'
    dst_images = dest_dir / split / 'images'
    dst_labels = dest_dir / split / 'labels'
    
    if not src_images.exists():
        print(f"  Warning: {src_images} not found")
        return stats
    
    dst_images.mkdir(parents=True, exist_ok=True)
    dst_labels.mkdir(parents=True, exist_ok=True)
    
    for img_path in src_images.glob('*'):
        if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
            new_name = f"roadlane_{img_path.name}"
            shutil.copy2(img_path, dst_images / new_name)
            stats['images'] += 1
            
            label_name = img_path.stem + '.txt'
            label_path = src_labels / label_name
            if label_path.exists():
                new_label_name = f"roadlane_{label_name}"
                shutil.copy2(label_path, dst_labels / new_label_name)
                stats['labels'] += 1
            else:
                stats['skipped'] += 1
    
    return stats


def copy_bdd100k_data(source_dir: Path, dest_dir: Path, split: str) -> Dict[str, int]:
    """Copy BDD100K data with class remapping."""
    stats = {'images': 0, 'labels': 0, 'skipped': 0, 'lane_only': 0}
    
    src_images = source_dir / split / 'images'
    src_labels = source_dir / split / 'labels'
    dst_images = dest_dir / split / 'images'
    dst_labels = dest_dir / split / 'labels'
    
    if not src_images.exists():
        print(f"  Warning: {src_images} not found")
        return stats
    
    dst_images.mkdir(parents=True, exist_ok=True)
    dst_labels.mkdir(parents=True, exist_ok=True)
    
    for img_path in src_images.glob('*'):
        if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
            label_name = img_path.stem + '.txt'
            label_path = src_labels / label_name
            
            if not label_path.exists():
                stats['skipped'] += 1
                continue
            
            new_name = f"bdd100k_{img_path.name}"
            new_label_name = f"bdd100k_{label_name}"
            
            if remap_bdd100k_label(label_path, dst_labels / new_label_name):
                shutil.copy2(img_path, dst_images / new_name)
                stats['images'] += 1
                stats['labels'] += 1
            else:
                stats['lane_only'] += 1
    
    return stats

In [ ]:
# Merge datasets
print("="*60)
print("MERGING DATASETS")
print("="*60)

start_time = time.time()

for split in ['train', 'valid', 'test']:
    print(f"\nProcessing {split} split...")
    
    # Road Lane
    print("  Copying Road Lane data...")
    rl_stats = copy_road_lane_data(ROAD_LANE_DIR, UNIFIED_DIR, split)
    print(f"    Images: {rl_stats['images']}, Labels: {rl_stats['labels']}, Skipped: {rl_stats['skipped']}")
    
    # BDD100K
    print("  Copying BDD100K data (with class remapping)...")
    bdd_stats = copy_bdd100k_data(BDD100K_DIR, UNIFIED_DIR, split)
    print(f"    Images: {bdd_stats['images']}, Labels: {bdd_stats['labels']}, "
          f"Skipped: {bdd_stats['skipped']}, Lane-only: {bdd_stats['lane_only']}")

merge_time = time.time() - start_time
print(f"\n✓ Dataset merge completed in {merge_time:.1f} seconds")

In [ ]:
# Create unified data.yaml
data_yaml_content = {
    'train': str(UNIFIED_DIR / 'train' / 'images'),
    'val': str(UNIFIED_DIR / 'valid' / 'images'),
    'test': str(UNIFIED_DIR / 'test' / 'images'),
    'nc': 17,
    'names': UNIFIED_CLASS_NAMES
}

UNIFIED_YAML = UNIFIED_DIR / 'data_unified.yaml'
with open(UNIFIED_YAML, 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print(f"Created: {UNIFIED_YAML}")
print(f"\nContents:")
with open(UNIFIED_YAML, 'r') as f:
    print(f.read())

In [ ]:
# Validate merged dataset
print("\n" + "="*60)
print("DATASET VALIDATION")
print("="*60)

class_counts = defaultdict(int)
total_images = 0
total_labels = 0

for split in ['train', 'valid', 'test']:
    images_dir = UNIFIED_DIR / split / 'images'
    labels_dir = UNIFIED_DIR / split / 'labels'
    
    if not images_dir.exists():
        continue
    
    n_images = len(list(images_dir.glob('*')))
    n_labels = len(list(labels_dir.glob('*.txt')))
    
    print(f"\n{split}:")
    print(f"  Images: {n_images}")
    print(f"  Labels: {n_labels}")
    
    total_images += n_images
    total_labels += n_labels
    
    # Count classes
    for label_path in labels_dir.glob('*.txt'):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    class_id = int(parts[0])
                    class_counts[class_id] += 1

print(f"\nTotal: {total_images} images, {total_labels} labels")

print("\nClass Distribution:")
print("-" * 40)
for class_id in sorted(class_counts.keys()):
    name = UNIFIED_CLASS_NAMES[class_id] if class_id < len(UNIFIED_CLASS_NAMES) else f"unknown_{class_id}"
    count = class_counts[class_id]
    category = "Lane" if class_id < 6 else "Traffic"
    print(f"  {class_id:2d} [{category:7s}] {name:15s}: {count:,}")

## **Step 2: Training Configuration**

In [ ]:
# Enhanced training configuration
TRAINING_CONFIG = {
    # Core settings
    'epochs': 100,           # More epochs for better convergence
    'imgsz': 640,            # Image size
    'batch': 16,             # Batch size (reduce if GPU OOM)
    'patience': 20,          # Early stopping patience
    'device': DEVICE,        # Training device
    'workers': 4,            # Data loader workers
    
    # Learning rate settings
    'lr0': 0.01,             # Initial learning rate
    'lrf': 0.01,             # Final learning rate (lr0 * lrf)
    'warmup_epochs': 3,      # Warmup epochs
    'warmup_momentum': 0.8,  # Warmup initial momentum
    'cos_lr': True,          # Cosine learning rate scheduler
    
    # Optimizer
    'optimizer': 'AdamW',    # AdamW optimizer
    'momentum': 0.937,       # SGD momentum/Adam beta1
    'weight_decay': 0.0005,  # Weight decay
    
    # Data augmentation
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,          # Disabled for driving scenes
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
    
    # Output
    'save': True,
    'save_period': 10,
    'plots': True,
    'verbose': True,
}

print("Training Configuration")
print("="*50)
for key in ['epochs', 'imgsz', 'batch', 'lr0', 'cos_lr', 'optimizer']:
    print(f"  {key}: {TRAINING_CONFIG[key]}")

## **Step 3: Train YOLO11n on Unified Dataset**

In [ ]:
# Load pretrained YOLO11n
print('Loading pretrained YOLO11n...')
model_n = YOLO('yolo11n.pt')
model_n.info()

In [ ]:
# Train YOLO11n
print('='*60)
print('Training YOLO11n on Unified Dataset (17 classes)')
print('='*60)

start_time = time.time()

results_n = model_n.train(
    data=str(UNIFIED_YAML),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_unified',
    exist_ok=True,
    **TRAINING_CONFIG
)

training_time_n = time.time() - start_time
print(f'\n✓ YOLO11n training completed in {training_time_n/3600:.2f} hours')

## **Step 4: Train YOLO11l on Unified Dataset**

In [ ]:
# Load pretrained YOLO11l
print('Loading pretrained YOLO11l...')
model_l = YOLO('yolo11l.pt')
model_l.info()

In [ ]:
# Train YOLO11l (reduced batch size)
print('='*60)
print('Training YOLO11l on Unified Dataset (17 classes)')
print('='*60)

config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8  # Reduced for larger model

start_time = time.time()

results_l = model_l.train(
    data=str(UNIFIED_YAML),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_unified',
    exist_ok=True,
    **config_l
)

training_time_l = time.time() - start_time
print(f'\n✓ YOLO11l training completed in {training_time_l/3600:.2f} hours')

## **Step 5: Validate & Compare Models**

In [ ]:
def evaluate_model(model_path, dataset_yaml, model_name):
    """Evaluate model and return metrics."""
    model = YOLO(str(model_path))
    
    print(f"\n{'='*60}")
    print(f"Evaluating {model_name}")
    print(f"{'='*60}")
    
    metrics = model.val(data=str(dataset_yaml), split='test', verbose=False)
    
    # Overall
    print(f"\nmAP50:     {metrics.box.map50:.4f}")
    print(f"mAP50-95:  {metrics.box.map:.4f}")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall:    {metrics.box.mr:.4f}")
    
    # Per-category
    ap50 = metrics.box.ap50
    lane_map = np.mean([ap50[i] for i in LANE_CLASSES if i < len(ap50)])
    traffic_map = np.mean([ap50[i] for i in TRAFFIC_CLASSES if i < len(ap50)])
    
    print(f"\nLane Types mAP50:     {lane_map:.4f}")
    print(f"Traffic Objects mAP50: {traffic_map:.4f}")
    
    return {
        'model': model_name,
        'map50': metrics.box.map50,
        'map': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr,
        'lane_map50': lane_map,
        'traffic_map50': traffic_map
    }

In [ ]:
# Evaluate both models
results = []

n_path = RUNS_DIR / 'finetune' / 'yolo11n_unified' / 'weights' / 'best.pt'
l_path = RUNS_DIR / 'finetune' / 'yolo11l_unified' / 'weights' / 'best.pt'

if n_path.exists():
    results.append(evaluate_model(n_path, UNIFIED_YAML, 'YOLO11n Unified'))

if l_path.exists():
    results.append(evaluate_model(l_path, UNIFIED_YAML, 'YOLO11l Unified'))

# Summary table
if results:
    df = pd.DataFrame(results)
    print("\n" + "="*80)
    print("MODEL COMPARISON SUMMARY")
    print("="*80)
    display(df)

## **Step 6: Visual Inference Test**

In [ ]:
# Test inference
test_dir = UNIFIED_DIR / 'test' / 'images'
best_model = YOLO(str(n_path)) if n_path.exists() else None

if best_model and test_dir.exists():
    sample_images = list(test_dir.glob('*.jpg'))[:6]
    
    if sample_images:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(sample_images):
            results = best_model.predict(str(img_path), verbose=False, conf=0.25)
            annotated = results[0].plot()
            annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            axes[idx].imshow(annotated)
            source = "Lane" if 'roadlane' in img_path.name else "Traffic"
            axes[idx].set_title(f"[{source}] {img_path.name[:25]}...", fontsize=9)
            axes[idx].axis('off')
            
            # Count detections
            if results[0].boxes is not None:
                classes = results[0].boxes.cls.cpu().numpy().astype(int)
                lane_cnt = sum(1 for c in classes if c < 6)
                traffic_cnt = sum(1 for c in classes if c >= 6)
                print(f"{img_path.name}: Lanes={lane_cnt}, Traffic={traffic_cnt}")
        
        plt.tight_layout()
        plt.savefig(str(RUNS_DIR / 'inference_samples.png'), dpi=150)
        plt.show()

## **Step 7: Save Models**

In [ ]:
# Copy best models to output
output_dir = Path('/kaggle/working/models')
output_dir.mkdir(exist_ok=True)

for src, name in [(n_path, 'yolo11n_unified_best.pt'), (l_path, 'yolo11l_unified_best.pt')]:
    if src.exists():
        shutil.copy2(src, output_dir / name)
        print(f"✓ Saved: {output_dir / name}")

print(f"\nModels saved to: {output_dir}")
print("Download these from the Output tab.")

## **Training Complete! 🎉**

The unified models detect:
- ✅ **6 lane types**: divider, dotted, double, random, road-sign, solid
- ✅ **11 traffic objects**: bike, bus, car, drivable area, motor, person, rider, traffic light, traffic sign, train, truck